In [1]:
1+1

2

In [8]:
import os
%pwd

'e:\\UDemy Final\\MLOPS\\TextSummarizer\\research'

In [9]:
os.chdir("../")
%pwd

'e:\\UDemy Final\\MLOPS\\TextSummarizer'

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [10]:
%pwd

'e:\\UDemy Final\\MLOPS\\TextSummarizer'

In [12]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
            
        )

        return model_evaluation_config

In [15]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk

import torch
import pandas as pd
from tqdm import tqdm

In [ ]:
import os
from pathlib import Path
import pandas as pd
import torch
from tqdm import tqdm
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i:i + batch_size]

    def calculate_metric_on_test_ds(
        self,
        dataset,
        metric,
        model,
        tokenizer,
        batch_size=2,
        device="cpu",
        column_text="dialogue",
        column_summary="summary",
    ):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
            inputs = tokenizer(
                article_batch,
                max_length=1024,
                truncation=True,
                padding=True,
                return_tensors="pt"
            )

            summaries = model.generate(
                input_ids=inputs["input_ids"].to(device),
                attention_mask=inputs["attention_mask"].to(device),
                length_penalty=0.8,
                num_beams=8,
                max_length=128
            )

            decoded_summaries = [
                tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                for s in summaries
            ]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        return metric.compute()

    def _load_model_and_tokenizer(self, device):
        # Normalize local paths so relative values from config do not get interpreted as HF repo ids
        project_root = Path.cwd()
        if not (project_root / "config" / "config.yaml").exists():
            candidate = project_root.parent
            if (candidate / "config" / "config.yaml").exists():
                project_root = candidate

        model_path = Path(self.config.model_path)
        tokenizer_path = Path(self.config.tokenizer_path)

        if not model_path.is_absolute():
            model_path = project_root / model_path
        if not tokenizer_path.is_absolute():
            tokenizer_path = project_root / tokenizer_path

        model_path = model_path.expanduser().resolve()
        tokenizer_path = tokenizer_path.expanduser().resolve()

        print(f"Resolved model_path: {model_path}")
        print(f"Resolved tokenizer_path: {tokenizer_path}")

        # Only treat as local if both directories exist
        if model_path.exists() and tokenizer_path.exists():
            try:
                print(f"Tentando modelo local: {model_path}")
                tokenizer = AutoTokenizer.from_pretrained(str(tokenizer_path), local_files_only=True)
                model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(str(model_path), local_files_only=True).to(device)
                print("Modelo local carregado com sucesso.")
                return model_pegasus, tokenizer, "local"
            except Exception as e:
                print(f"Falha ao carregar modelo local ({type(e).__name__}): {e}")
        else:
            print(f"Modelo/tokenizer local não encontrado. model_exists={model_path.exists()} tokenizer_exists={tokenizer_path.exists()}")

        print("Fallback para modelo base: google/pegasus-cnn_dailymail")
        base_model = "google/pegasus-cnn_dailymail"
        tokenizer = AutoTokenizer.from_pretrained(base_model)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(base_model).to(device)
        return model_pegasus, tokenizer, "hf_base"

    def evaluate(self):
        device = "cpu"

        dataset_samsum_pt = load_from_disk(str(self.config.data_path))
        rouge_metric = evaluate.load("rouge")

        model_pegasus, tokenizer, model_source = self._load_model_and_tokenizer(device=device)

        test_sample = dataset_samsum_pt["test"].select(range(10))

        score = self.calculate_metric_on_test_ds(
            dataset=test_sample,
            metric=rouge_metric,
            model=model_pegasus,
            tokenizer=tokenizer,
            batch_size=2,
            device=device,
            column_text="dialogue",
            column_summary="summary"
        )

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_dict = {rn: score[rn] for rn in rouge_names}

        print("Model source:", model_source)
        print("ROUGE:", rouge_dict)

        metric_path = Path(self.config.metric_file_name)
        metric_path.parent.mkdir(parents=True, exist_ok=True)

        df = pd.DataFrame(rouge_dict, index=["pegasus"])
        df.to_csv(metric_path, index=False)
        print("Métricas salvas em:", metric_path)


In [ ]:
from pathlib import Path

project_root = Path("/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer")

config = ConfigurationManager()
model_evaluation_config = config.get_model_evaluation_config()

model_evaluation_config.data_path = project_root / "artifacts" / "data_transformation" / "samsum_dataset"
model_evaluation_config.model_path = project_root / "artifacts" / "model_trainer" / "pegasus-samsum-model"
model_evaluation_config.tokenizer_path = project_root / "artifacts" / "model_trainer" / "tokenizer"
model_evaluation_config.metric_file_name = project_root / "artifacts" / "model_evaluation" / "metrics.csv"

print("data_path:", model_evaluation_config.data_path, model_evaluation_config.data_path.exists())
print("model_path:", model_evaluation_config.model_path, model_evaluation_config.model_path.exists())
print("tokenizer_path:", model_evaluation_config.tokenizer_path, model_evaluation_config.tokenizer_path.exists())

model_evaluation = ModelEvaluation(config=model_evaluation_config)
model_evaluation.evaluate()
